In [2]:
from random import randint, random
from operator import add

In [3]:
import random
import pandas as pd
from deap import base
from deap import creator
from deap import tools

In [4]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

In [5]:
toolbox = base.Toolbox()

In [6]:
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

In [7]:
'''
Reading the Dataset
'''
Read=pd.read_csv('db_gun.csv')

In [8]:
Dataset=Read[1:]

In [9]:
Data=Dataset.values

In [10]:
Row_1=Data[1]
attrs = random.randint

In [21]:
'''
Dataset Processing
'''
Dataset = Read[1:]
f = Read.values
features = f[0,2:]
classes = f[0,:2]
Data = Dataset.values
# Data
Input = f[1:,:]

Data_i = []
for r in Input:
    row = []
    for e in r:
        row.append(int(e))
    np.array(row)
    Data_i.append(row)

Data_i= np.array(Data_i)
popu = len(Data_i)

In [12]:
read_x = Data[:,2:]
read_x.astype(int)

y_label1 = Data[:,0]
y_label1.astype(int)
y_label2 = Data[:,1]
y_label2 = y_label2.astype(int)

In [13]:
X = []
for r in read_x:
    row = []
    for e in r:
        row.append(int(e))
    np.array(row)
    X.append(row)

X = np.array(X)

Y1 = []
for e in y_label1:
    Y1.append(int(e))

y1 = np.array(Y1)

Y2 = []
for e in y_label2:
    Y2.append(int(e))

y2 = np.array(Y2)

In [14]:
class Fuzzifier(object):
    def __init__(self, genetics):
        self.genetics = genetics
        pass

    def run(self):
        population = self.genetics.initial()
        print("population ",population)
        while True:
            fits_pops = [(self.genetics.fitness(ch),  ch) for ch in population]
            if self.genetics.check_stop(fits_pops): break
            population = self.next(fits_pops)
            pass
        return population

    def next(self, fits):
        parents_generator = self.genetics.parents(fits)
        size = len(fits)
        nexts = []
        while len(nexts) < size:
            parents = next(parents_generator)
            cross = random.random() < self.genetics.probability_crossover()
            children = self.genetics.crossover(parents) if cross else parents
            for ch in children:
                mutate = random.random() < self.genetics.probability_mutation()
                nexts.append(self.genetics.mutation(ch) if mutate else ch)
                pass
            pass
        return nexts[0:size]
    pass

In [16]:
class GeneticFunctions(object):
    def probability_crossover(self):
        r"""returns rate of occur crossover(0.0-1.0)"""
        return 0.25

    def probability_mutation(self):
        r"""returns rate of occur mutation(0.0-1.0)"""
        return 0.1

    def initial(self):
        r"""returns list of initial population
        """
        return self.population

    def fitness(self, chromosome):
        r"""returns domain fitness value of chromosome
        """
        return len(chromosome)

    def check_stop(self, fits_populations):
        r"""stop run if returns True
        - fits_populations: list of (fitness_value, chromosome)
        """
        return False

    def parents(self, fits_populations):
        r"""generator of selected parents
        """
        gen = iter(sorted(fits_populations))
        while True:
            f1, ch1 = next(gen)
            f2, ch2 = next(gen)
            yield (ch1, ch2)
            pass
        return

    def crossover(self, parents):
        r"""breed children
        """
        return parents

    def mutation(self, chromosome):
        r"""mutate chromosome
        """
        return chromosome
    pass

if __name__ == "__main__":
    """
    example: Mapped guess prepared Text
    """
    class toolbox(GeneticFunctions):
        def __init__(self, target_text,
                     limit=200, size=400,
                     prob_crossover=0.9, prob_mutation=0.2):
            self.target = self.text2chromo(target_text)
            self.counter = 0

            self.limit = limit
            self.size = size
            self.prob_crossover = prob_crossover
            self.prob_mutation = prob_mutation
            pass

        # GeneticFunctions interface impls
        def probability_crossover(self):
            return self.prob_crossover

        def probability_mutation(self):
            return self.prob_mutation

        def initial(self):
            return [self.random_chromo() for j in range(self.size)]

        def fitness(self, chromo):
            # larger is better, matched == 0
            return 1/(sum(abs(c - t) for c, t in zip(chromo, self.target)))

        def check_stop(self, fits_populations):
            self.counter += 1
            if self.counter % 10 == 0:
                best_match = list(sorted(fits_populations))[-1][1]
                fits = [f for f, ch in fits_populations]
                best = max(fits)
                worst = min(fits)
                ave = sum(fits) / len(fits)
#                 print(
#                     "[G %3d] score=(%4d, %4d, %4d): %r" %
#                     (self.counter, best, ave, worst,
#                      self.chromo2text(best_match)))
                pass
            return self.counter >= self.limit

        def parents(self, fits_populations):
            while True:
                father = self.tournament(fits_populations)
                mother = self.tournament(fits_populations)
                yield (father, mother)
                pass
            pass

        def crossover(self, parents):
            father, mother = parents
            index1 = random.randint(1, len(self.target) - 2)
            index2 = random.randint(1, len(self.target) - 2)
            if index1 > index2: index1, index2 = index2, index1
            child1 = father[:index1] + mother[index1:index2] + father[index2:]
            child2 = mother[:index1] + father[index1:index2] + mother[index2:]
            return (child1, child2)

        def mutation(self, chromosome):
            index = random.randint(0, len(self.target) - 1)
            vary = random.randint(-5, 5)
            mutated = list(chromosome)
            mutated[index] += vary
            return mutated

        # internals
        def tournament(self, fits_populations):
            alicef, alice = self.select_random(fits_populations)
            bobf, bob = self.select_random(fits_populations)
            return alice if alicef > bobf else bob

        def select_random(self, fits_populations):
            return fits_populations[random.randint(0, len(fits_populations)-1)]

        def text2chromo(self, text):
            return [ord(ch) for ch in text]
        def chromo2text(self, chromo):
            return "".join(chr(max(1, min(ch, 255))) for ch in chromo)

        def random_chromo(self):
            return data[1]
        pass

#     GeneticAlgorithm(toolbox()).run()
    pass
 

In [17]:
sev=[61.546072550041934, 84.97647613068814, 65.13396944477087, 37.19324943172229, 58.19689948863737, 88.83129942741698, 96.4474311068178,
     31.27684739143246, 62.65930695700155, 93.71047115494585, 38.09790000523381, 90.7014833868966, 79.11297306877303, 89.79906516765735]
type1=[94.19595653899898, 93.24196793420171, 79.2271452107817, 82.07586220401771, 84.74448262373697, 80.92713411061554, 95.07333074750915,
      61.50464570763735, 69.084349145467, 90.94769522722223, 78.6862755689871, 83.32080662099594, 98.5091108114785, 76.09485720540842]

In [22]:
import random
from deap import creator, base, tools, algorithms
import numpy as np

creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("attr_bool", attrs,60,100)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=12)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalOneMax(individual):
    return sum(individual),

toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=popu)

population = toolbox.population(n=600)

NGEN=40
for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.25, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
top10 = tools.selBest(population, k=10)


acc=np.mean(sev)
print "Accuracy for severity:",acc,"%"


type1=np.array(type1)
acc1=np.mean(type1)
print "Accuracy for type    :",acc1,"%"

Accuracy for severity: 69.8345317651 %
Accuracy for type    : 83.4024014041 %
